In [ ]:
##newest 06/03 !!!!final

import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
# import pyodbc
# import random
# import zipfile
# import io
import re
# import csv
from tqdm import tqdm  # Import tqdm for progress tracking

reader_ICD = ThunderReader('/home/bram/internal_expansion/HOSPITALS/MGB/ICD')
all_keys_ICD = list(reader_ICD.keys())
print(all_keys_ICD)

ICDPL_dfs = []

# Define the regex patterns for ICD9 and ICD10 codes using non-capturing groups
icd9_pattern = re.compile(r'(?:^|,)\s*(3320|332\.0)\s*(?:,|$)', re.IGNORECASE)
icd10_pattern = re.compile(r'(?:^|,)\s*G20(?:\s*|,|$)', re.IGNORECASE)

# Loop through the partitions and process the data
for i in tqdm(range(1, len(all_keys_ICD) + 1)):
    ICDPL_df = reader_ICD[f'ICD_partition_{i}']
    
    # Strip leading/trailing spaces and convert to string
    ICDPL_df['ICDCD'] = ICDPL_df['ICDCD'].astype(str).str.strip()
    
    # Filter with regex pattern for combined ICD9 and ICD10 codes
    icd9_matches = ICDPL_df['ICDCD'].str.contains(icd9_pattern)
    icd10_matches = ICDPL_df['ICDCD'].str.contains(icd10_pattern)
    
    # Include rows that match either ICD9 or ICD10 patterns
    included_df = ICDPL_df[icd9_matches | icd10_matches]
    
    ICDPL_dfs.append(included_df)

# Concatenate all DataFrames
ICDPL_df = pd.concat(ICDPL_dfs, axis=0, ignore_index=True)

# Print the unique 'ICDCD' values and the DataFrame
print(ICDPL_df['ICDCD'].unique())
print(ICDPL_df)

In [ ]:
#define timewindow
start_date = pd.Timestamp('2016-05-01')
end_date = pd.Timestamp('2023-01-01')

ICDPL_df['ShiftedContactDTS'] = pd.to_datetime(ICDPL_df['ShiftedContactDTS'])

# FILTER BASED ON TIME WINDOW
filt_ICDPL_df = ICDPL_df[(ICDPL_df['ShiftedContactDTS'] >= start_date) & (ICDPL_df['ShiftedContactDTS'] < end_date)]

print(filt_ICDPL_df['ICDCD'])
print(filt_ICDPL_df)

In [ ]:
# GETCSVOFALLPOSDATA

filt_ICDPL_df = filt_ICDPL_df.rename(columns={'ShiftedContactDTS': 'CreateDate'})

filt_ICDPL_df.to_csv('MGB_ICDPLUS.csv', index=False)

In [ ]:
unique_icdcd = filt_ICDPL_df['ICDCD'].unique()
print(list(unique_icdcd))
print(filt_ICDPL_df)

In [ ]:
unique_ids_ICD_plus = set(filt_ICDPL_df['BDSPPatientID'].unique())

In [ ]:
print(len(unique_ids_ICD_plus))
ICDPL_MGB_LIST= list(unique_ids_ICD_plus)

ICDPL_MGB_df = pd.DataFrame(ICDPL_MGB_LIST, columns=['BDSPPatientID'])
print(len(ICDPL_MGB_df))
ICDPL_MGB_df.to_csv('ICDPL_MGB_New.csv', index=False)

In [ ]:
import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
import pyodbc
import random
import zipfile
import io
import re
import csv
from tqdm import tqdm  # Import tqdm for progress tracking

reader_ICD = ThunderReader('/home/bram/internal_expansion/MGB/ICD')
all_keys_ICD = list(reader_ICD.keys())
print(all_keys_ICD)


## icdM

ICDMN_dfs = []

pattern = re.compile(r'^(332\.?0|G20|G20|G20\.A|G20\.A1|G20\.A2|G20\.B|G20\.B1|G20\.B2)$', re.IGNORECASE)  # pattern = re.compile(r'^(332\.?0|G20)$', re.IGNORECASE)
# G20|G20\.A|G20\.A1|G20\.A2|G20\.B|G20\.B1|G20\.B2
for i in tqdm(range (1, len(all_keys_ICD) + 1)):
    ICDMN_df = reader_ICD[f'ICD_partition_{i}']
    
    # Debugging: Print unique DiagnosisCode values
    # print(f"Unique DiagnosisCode values in partition {i}:")
    # print(ICDPL_df['DiagnosisCode'].unique())
    
    # Strip leading/trailing spaces and convert to string
    ICDMN_df['ICDCD'] = ICDMN_df['ICDCD'].astype(str).str.strip()
    
    # Filter with regex pattern
    ICDMN_df = ICDMN_df[~ICDMN_df.ICDCD.str.match(pattern)]
    
    ICDMN_dfs.append(ICDMN_df)

ICDMN_df = pd.concat(ICDMN_dfs, axis=0, ignore_index=True)

# # Print the 'DiagnosisCode' column
print(ICDMN_df['ICDCD'].unique())
print(ICDMN_df)

In [ ]:
import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
import pyodbc
import random
import zipfile
import io
import re
import csv
from tqdm import tqdm  # Import tqdm for progress tracking

reader_ICD = ThunderReader('/home/bram/internal_expansion/MGB/ICD')
all_keys_ICD = list(reader_ICD.keys())
print(all_keys_ICD)


## icdmn

ICDMN_dfs = []

pattern = re.compile(r'^(332\.?0|G20|G20|G20\.A|G20\.A1|G20\.A2|G20\.B|G20\.B1|G20\.B2)$', re.IGNORECASE)

chunk_size = 50  # Adjust chunk size based on your system's memory capacity

# Loop through each partition in chunks and process DataFrames
for i in tqdm(range(0, len(all_keys_ICD), chunk_size), desc="Processing ICD partitions"):
    chunk_keys = all_keys_ICD[i:i + chunk_size]
    
    chunk_dfs = []
    for key in chunk_keys:
        ICDMN_df = reader_ICD[key]
        
        # Strip leading/trailing spaces and convert to string
        ICDMN_df['ICDCD'] = ICDMN_df['ICDCD'].astype(str).str.strip()
        
        # Filter with regex pattern
        ICDMN_df = ICDMN_df[~ICDMN_df.ICDCD.str.match(pattern)]
        
        chunk_dfs.append(ICDMN_df)
    
    # Concatenate the chunk DataFrames into a single DataFrame
    chunk_df = pd.concat(chunk_dfs, axis=0, ignore_index=True)
    
    # Append the concatenated chunk DataFrame to the list
    ICDMN_dfs.append(chunk_df)
    
    # Clear chunk_dfs to free up memory
    del chunk_dfs

# Concatenate all processed DataFrames into a single DataFrame
ICDMN_df = pd.concat(ICDMN_dfs, axis=0, ignore_index=True)

# Print the unique 'ICDCD' values
print(ICDMN_df['ICDCD'].unique())
print(ICDMN_df)

In [ ]:
print(ICDMN_df)

In [ ]:
ICDMN_df['ShiftedContactDTS'] = pd.to_datetime(ICDMN_df['ShiftedContactDTS'])

# FILTER BASED ON TIME WINDOW
filt_ICDMN_df = ICDMN_df[(ICDMN_df['ShiftedContactDTS'] >= start_date) & (ICDMN_df['ShiftedContactDTS'] < end_date)]

print(filt_ICDMN_df['ICDCD'])
print(filt_ICDMN_df)

In [ ]:
import pandas as pd
from thunderpack import ThunderReader  # install thunderpack if not yet: pip install thunderpack
import pandas as pd
import pyodbc
import random
import zipfile
import io
import re 
import csv

In [ ]:
reader_ICD = ThunderReader('/home/bram/internal_expansion/MGB/ICD')
all_keys_ICD = list(reader_ICD.keys())
print(all_keys_ICD)

In [ ]:
### count total number of files

# Initialize the list to collect DataFrames
ICD_dfs_initial = []

# Loop through each partition and collect DataFrames
for i in range(1, len(all_keys_ICD) + 1):
    ICD_df = reader_ICD[f'ICD_partition_{i}']
    
    # Append the DataFrame to the list
    ICD_dfs_initial.append(ICD_df)

# Concatenate all initial DataFrames into a single DataFrame
ICD_df_initial = pd.concat(ICD_dfs_initial, axis=0, ignore_index=True)

# Count the total number of rows in the initial data
total_rows_initial = ICD_df_initial.shape[0]

# Print the total number of rows
print(f"Total number of rows in the initial data: {total_rows_initial}")
print(ICD_df_initial.columns())

print(ICD_df_initial.head())



In [ ]:
# unique_combinations_icd10 = ICD_df_initial['icd10_code'].str.contains('332\.?0?0', case=False, na=False)
# unique_combinations_icd10 = ICD_df_initial[unique_combinations_icd10]['icd10_code'].unique()

# unique_combinations_icd9 = ICD_df_initial['icd9_code'].str.contains('G20', case=False, na=False)
# unique_combinations_icd9 = ICD_df_initial[unique_combinations_icd9]['icd9_code'].unique()

# # Print the unique combinations
# print(unique_combinations_icd10)
# print(unique_combinations_icd9)